In [1]:
import numpy as np
import os
import pandas as pd
import arcpy
# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()
from arcgis.geoanalytics import manage_data

from IPython.display import display, Markdown

In [2]:
dirWork         = os.getcwd()
dirInput        = os.path.join(dirWork , r'input'                  )
dirResults      = os.path.join(dirWork , r'results'                ) # This is the folder where all the results will be saved
dirATOdata      = os.path.join(dirInput, r'TDM_ATO_Output_RTP_v900') # This folder should contain a copy of all the TDM ATO outputs from the latest official version

print("Working Directory: "      + dirWork   )
print("Input Directory: "        + dirInput  )
print("Results Directory: "      + dirResults)

# This should return filename for TDM output that is copied into the input data
def getCSVFilename(year):
    return os.path.join(dirATOdata, 'Access_to_Opportunity_' + str(year) + '.csv')

#list of anlysis years
listYear     = [2023,2032,2042,2050,2050]

highestTazID = 3546

Working Directory: e:\GitHub\ATO-Web-App\_dataprep
Input Directory: e:\GitHub\ATO-Web-App\_dataprep\input
Results Directory: e:\GitHub\ATO-Web-App\_dataprep\results


In [3]:
dfATOCombined = pd.DataFrame()

for year in sorted(list(set(listYear))):
    
    dfATO = pd.read_csv(getCSVFilename(year))
    
    dfATO = dfATO[dfATO['TAZID']<=highestTazID].copy()

    dfATO.rename(columns={'HH'             :'HH'         ,
                          'Job'            :'JOB'        ,
                          'JobByAuto'      :'JOBAUTO'    ,
                          'HHByAuto'       :'HHAUTO'     ,
                          'CompJobHHByAuto':'COMPAUTO'   ,
                          'JobByTran'      :'JOBTRANSIT' ,
                          'HHByTran'       :'HHTRANSIT'  ,
                          'CompJobHHByTran':'COMPTRANSIT'}, inplace=True)

    dfATO = dfATO[['TAZID','HH','JOB','HHAUTO','JOBAUTO','COMPAUTO','HHTRANSIT','JOBTRANSIT','COMPTRANSIT']]

    dfATO['YEAR'] = year-2000

    dfATOCombined = pd.concat([dfATOCombined, dfATO])

#dfATOCombined.columns
dfATOCombined

,TAZID,HH,JOB,HHAUTO,JOBAUTO,COMPAUTO,HHTRANSIT,JOBTRANSIT,COMPTRANSIT,YEAR
0,1,0.0,0.0,26727,45090,0,0,0,0,23
1,2,13.0,0.0,27703,46719,46719,0,0,0,23
2,3,13.5,0.0,35466,60407,60407,0,0,0,23
3,4,3.3,0.0,33039,56355,56355,0,0,0,23
4,5,53.5,5.0,29762,50154,49225,0,0,0,23
...,...,...,...,...,...,...,...,...,...,...
3541,3542,2.0,0.0,40280,48159,48159,0,0,0,50
3542,3543,0.0,0.0,21656,24964,0,0,0,0,50
3543,3544,0.0,0.0,35347,45643,0,0,0,0,50
3544,3545,5.0,0.0,16800,22029,22029,0,0,0,50


In [4]:
dfATOCombined_melt = dfATOCombined.melt(id_vars=('TAZID','YEAR'),value_vars=('HH','JOB','HHAUTO','JOBAUTO','COMPAUTO','HHTRANSIT','JOBTRANSIT','COMPTRANSIT'))
dfATOCombined_melt['variable'] = dfATOCombined_melt['variable'] +'_' + dfATOCombined_melt['YEAR'].astype(str)
dfATOCombined_melt.drop(columns=('YEAR'))
dfATOCombined_melt_xt = dfATOCombined_melt.pivot_table(index='TAZID', columns='variable', values='value')
dfATOCombined_melt_xt.reset_index(inplace=True)
dfATOCombined_melt_xt

variable,TAZID,COMPAUTO_23,COMPAUTO_32,COMPAUTO_42,COMPAUTO_50,COMPTRANSIT_23,COMPTRANSIT_32,COMPTRANSIT_42,COMPTRANSIT_50,HHAUTO_23,...,JOBAUTO_42,JOBAUTO_50,JOBTRANSIT_23,JOBTRANSIT_32,JOBTRANSIT_42,JOBTRANSIT_50,JOB_23,JOB_32,JOB_42,JOB_50
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26727.0,...,51888.0,53063.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,46719.0,50620.0,53743.0,54954.0,0.0,0.0,0.0,0.0,27703.0,...,53743.0,54954.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,60407.0,66044.0,69976.0,70770.0,0.0,0.0,0.0,0.0,35466.0,...,69976.0,70770.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,56355.0,61621.0,65317.0,66093.0,0.0,0.0,0.0,0.0,33039.0,...,65317.0,66093.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,49225.0,53109.0,56424.0,57822.0,0.0,0.0,0.0,0.0,29762.0,...,57666.0,58931.0,0.0,0.0,0.0,0.0,5.0,7.3,8.9,10.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3541,3542,46077.0,49730.0,51222.0,48159.0,0.0,0.0,0.0,0.0,26602.0,...,51222.0,48159.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3542,3543,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14133.0,...,27341.0,24964.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3543,3544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20012.0,...,46538.0,45643.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3544,3545,17353.0,18413.0,22413.0,22029.0,0.0,0.0,0.0,0.0,9599.0,...,22413.0,22029.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Replace 'OpenDataKey' with the actual item ID
item_id = "2fec557533dd4d9b9ccbd6f4c81ec870"

print("Downloading...")

# Get the item
item = gis.content.get(item_id)

# Access the first layer of the item
flayer = item.layers[0]

# Convert the feature layer to a Spatial DataFrame
sdfTAZ = pd.DataFrame.spatial.from_layer(flayer)

# Display the first few rows of the DataFrame
display(sdfTAZ)

Downloading...


,ACRES,ADJ_XY,CBD,CITY_FIPS,CITY_NAME,CITY_UGRC,CO_FIPS,CO_IDX,CO_NAME,CO_TAZID,...,SORT,SUBAREAID,Shape__Area,Shape__Length,TAZID,TAZID_V832,TERMTIME,WALK100,X,Y
0,53.885304,0,0,67000,Salt Lake City,SLC,35,324,SALT LAKE,350324,...,801,1,3.804613e+05,3019.727799,1229,1124,0,0,424640.657110,4.510008e+06
1,602.262229,0,0,7690,Bountiful,BTF,11,248,DAVIS,110248,...,401,1,4.272924e+06,9532.134381,829,613,0,0,428293.205659,4.527861e+06
2,53.680783,0,0,67000,Salt Lake City,SLC,35,325,SALT LAKE,350325,...,802,1,3.790177e+05,3018.051462,1230,1125,0,0,424882.550810,4.510006e+06
3,107.681660,0,0,67000,Salt Lake City,SLC,35,326,SALT LAKE,350326,...,803,1,7.602964e+05,3658.859273,1231,1126,0,0,425245.216636,4.510002e+06
4,107.325984,0,0,67000,Salt Lake City,SLC,35,327,SALT LAKE,350327,...,804,1,7.577864e+05,3655.720078,1232,1127,0,0,425728.287851,4.509998e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3541,222.080694,0,0,72280,Springville,SPV,49,867,UTAH,490867,...,2832,1,1.541132e+06,5131.057808,3083,2518,0,0,446602.419710,4.444526e+06
3542,259.885069,0,0,0,Spanish Fork,na,49,926,UTAH,490926,...,2833,1,1.802999e+06,6088.973755,3142,2521,0,0,445403.446563,4.443542e+06
3543,71.913180,0,0,72280,Springville,SPV,49,848,UTAH,490848,...,2834,1,4.991795e+05,3053.510595,3064,2525,0,0,447749.409836,4.445548e+06
3544,98.791359,0,0,72280,Springville,SPV,49,849,UTAH,490849,...,2835,1,6.857436e+05,3314.666935,3065,2526,0,0,448305.522557,4.445492e+06


In [6]:
### MANUALLY DOWNLOAD CSV AND THEN LOAD
# https://docs.google.com/spreadsheets/d/1R6hsy0HQoZH-zj1JuV5OU5neATdOwgTJWXNvHIiIDMs/edit#gid=1708662420
dfSubArea = pd.read_csv(os.path.join(dirInput,'CitySubAreaAssignments - cityareasum.csv'))
dfSubArea.drop(columns=('FREQUENCY'), inplace=True)
dfSubArea.rename(columns={'SUBAREA':'SMALLAREA'}, inplace=True)
dfSubArea

,CITY_NAME,SMALLAREA
0,Box Elder County North,Box Elder WFRC
1,Box Elder County South,Box Elder WFRC
2,Brigham City,Box Elder WFRC
3,Perry,Box Elder WFRC
4,Willard,Box Elder WFRC
...,...,...
105,Utah Lake,NaN
106,Weber County East Mountains,NaN
107,West Mountain,NaN
108,West Mountain Town,NaN


In [7]:
sdfTAZAll = pd.DataFrame.merge(sdfTAZ, dfSubArea, on='CITY_NAME', how='left')
sdfTAZAll.fillna('')
sdfTAZAll

,ACRES,ADJ_XY,CBD,CITY_FIPS,CITY_NAME,CITY_UGRC,CO_FIPS,CO_IDX,CO_NAME,CO_TAZID,...,SUBAREAID,Shape__Area,Shape__Length,TAZID,TAZID_V832,TERMTIME,WALK100,X,Y,SMALLAREA
0,53.885304,0,0,67000,Salt Lake City,SLC,35,324,SALT LAKE,350324,...,1,3.804613e+05,3019.727799,1229,1124,0,0,424640.657110,4.510008e+06,Salt Lake County North
1,602.262229,0,0,7690,Bountiful,BTF,11,248,DAVIS,110248,...,1,4.272924e+06,9532.134381,829,613,0,0,428293.205659,4.527861e+06,Davis County South
2,53.680783,0,0,67000,Salt Lake City,SLC,35,325,SALT LAKE,350325,...,1,3.790177e+05,3018.051462,1230,1125,0,0,424882.550810,4.510006e+06,Salt Lake County North
3,107.681660,0,0,67000,Salt Lake City,SLC,35,326,SALT LAKE,350326,...,1,7.602964e+05,3658.859273,1231,1126,0,0,425245.216636,4.510002e+06,Salt Lake County North
4,107.325984,0,0,67000,Salt Lake City,SLC,35,327,SALT LAKE,350327,...,1,7.577864e+05,3655.720078,1232,1127,0,0,425728.287851,4.509998e+06,Salt Lake County North
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3541,222.080694,0,0,72280,Springville,SPV,49,867,UTAH,490867,...,1,1.541132e+06,5131.057808,3083,2518,0,0,446602.419710,4.444526e+06,Utah County South
3542,259.885069,0,0,0,Spanish Fork,na,49,926,UTAH,490926,...,1,1.802999e+06,6088.973755,3142,2521,0,0,445403.446563,4.443542e+06,Utah County South
3543,71.913180,0,0,72280,Springville,SPV,49,848,UTAH,490848,...,1,4.991795e+05,3053.510595,3064,2525,0,0,447749.409836,4.445548e+06,Utah County South
3544,98.791359,0,0,72280,Springville,SPV,49,849,UTAH,490849,...,1,6.857436e+05,3314.666935,3065,2526,0,0,448305.522557,4.445492e+06,Utah County South


In [8]:
sdfTAZAll.rename(columns={'CITY_NAME':'CITYAREA'},inplace=True)

sdfTAZAll = sdfTAZAll[['TAZID','CO_TAZID','DEVACRES','CITYAREA','SMALLAREA','CO_FIPS','CO_NAME','SHAPE']]

sdfTAZATOforWFRCDataPortal = pd.DataFrame.merge(sdfTAZAll,dfATOCombined_melt_xt,on="TAZID")

sdfTAZATOforWFRCDataPortal = sdfTAZATOforWFRCDataPortal[['TAZID','CO_TAZID','DEVACRES','CITYAREA','SMALLAREA','CO_FIPS','CO_NAME',
                                                         'HH_23'         ,
                                                         'JOB_23'        ,
                                                         'JOBAUTO_23'    ,
                                                         'HHAUTO_23'     ,
                                                         'COMPAUTO_23'   ,
                                                         'JOBTRANSIT_23' ,
                                                         'HHTRANSIT_23'  ,
                                                         'COMPTRANSIT_23',
                                                         'HH_32'         ,
                                                         'JOB_32'        ,
                                                         'JOBAUTO_32'    ,
                                                         'HHAUTO_32'     ,
                                                         'COMPAUTO_32'   ,
                                                         'JOBTRANSIT_32' ,
                                                         'HHTRANSIT_32'  ,
                                                         'COMPTRANSIT_32',
                                                         'HH_42'         ,
                                                         'JOB_42'        ,
                                                         'JOBAUTO_42'    ,
                                                         'HHAUTO_42'     ,
                                                         'COMPAUTO_42'   ,
                                                         'JOBTRANSIT_42' ,
                                                         'HHTRANSIT_42'  ,
                                                         'COMPTRANSIT_42',
                                                         'HH_50'         ,
                                                         'JOB_50'        ,
                                                         'JOBAUTO_50'    ,
                                                         'HHAUTO_50'     ,
                                                         'COMPAUTO_50'   ,
                                                         'JOBTRANSIT_50' ,
                                                         'HHTRANSIT_50'  ,
                                                         'COMPTRANSIT_50',
                                                         'SHAPE'         ]]

sdfTAZATOforWFRCDataPortal

,TAZID,CO_TAZID,DEVACRES,CITYAREA,SMALLAREA,CO_FIPS,CO_NAME,HH_23,JOB_23,JOBAUTO_23,...,COMPTRANSIT_42,HH_50,JOB_50,JOBAUTO_50,HHAUTO_50,COMPAUTO_50,JOBTRANSIT_50,HHTRANSIT_50,COMPTRANSIT_50,SHAPE
0,1229,350324,53.885304,Salt Lake City,Salt Lake County North,35,SALT LAKE,194.8,460.5,563452.0,...,90156.0,437.8,470.7,647825.0,346909.0,527015.0,129943.0,47032.0,96656.0,"{'rings': [[[-12455792.2096342, 4974305.893117..."
1,829,110248,439.651427,Bountiful,Davis County South,11,DAVIS,376.7,0.0,254487.0,...,0.0,722.8,0.0,333409.0,207035.0,333409.0,0.0,0.0,0.0,"{'rings': [[[-12452338.7416187, 4998723.220743..."
2,1230,350325,53.680783,Salt Lake City,Salt Lake County North,35,SALT LAKE,79.3,1569.0,557030.0,...,73739.0,439.2,949.0,639996.0,342252.0,469056.0,131429.0,46757.0,82818.0,"{'rings': [[[-12455340.846539, 4973162.8833473..."
3,1231,350326,107.681660,Salt Lake City,Salt Lake County North,35,SALT LAKE,556.8,1577.5,543312.0,...,71866.0,677.2,1314.2,623556.0,333190.0,464534.0,122916.0,43592.0,79473.0,"{'rings': [[[-12454702.6418795, 4973151.769042..."
4,1232,350327,107.325984,Salt Lake City,Salt Lake County North,35,SALT LAKE,724.3,61.3,526076.0,...,98589.0,787.5,61.8,601170.0,322560.0,588166.0,107222.0,43156.0,104232.0,"{'rings': [[[-12454067.6619695, 4974307.454772..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3541,3083,490867,222.080694,Springville,Utah County South,49,UTAH,428.0,0.0,158764.0,...,14670.0,542.0,160.0,183306.0,134226.0,175673.0,17144.0,8086.0,15736.0,"{'rings': [[[-12426361.3999095, 4888336.383727..."
3542,3142,490926,257.286219,Spanish Fork,Utah County South,49,UTAH,0.0,192.0,161755.0,...,9165.0,1118.0,466.0,189657.0,136316.0,178649.0,11118.0,5359.0,9929.0,"{'rings': [[[-12427278.5433489, 4885975.922995..."
3543,3064,490848,71.913180,Springville,Utah County South,49,UTAH,186.0,146.0,157672.0,...,12232.0,265.0,146.0,167918.0,121564.0,156061.0,13118.0,7714.0,11736.0,"{'rings': [[[-12424511.3813815, 4888398.136459..."
3544,3065,490849,98.791359,Springville,Utah County South,49,UTAH,233.0,656.0,141528.0,...,9543.0,287.0,815.0,161254.0,117259.0,133132.0,12731.0,7441.0,9349.0,"{'rings': [[[-12424442.9205386, 4889379.844308..."


In [9]:
sdfTAZATOforWFRCDataPortal.columns

Index(['TAZID', 'CO_TAZID', 'DEVACRES', 'CITYAREA', 'SMALLAREA', 'CO_FIPS',
       'CO_NAME', 'HH_23', 'JOB_23', 'JOBAUTO_23', 'HHAUTO_23', 'COMPAUTO_23',
       'JOBTRANSIT_23', 'HHTRANSIT_23', 'COMPTRANSIT_23', 'HH_32', 'JOB_32',
       'JOBAUTO_32', 'HHAUTO_32', 'COMPAUTO_32', 'JOBTRANSIT_32',
       'HHTRANSIT_32', 'COMPTRANSIT_32', 'HH_42', 'JOB_42', 'JOBAUTO_42',
       'HHAUTO_42', 'COMPAUTO_42', 'JOBTRANSIT_42', 'HHTRANSIT_42',
       'COMPTRANSIT_42', 'HH_50', 'JOB_50', 'JOBAUTO_50', 'HHAUTO_50',
       'COMPAUTO_50', 'JOBTRANSIT_50', 'HHTRANSIT_50', 'COMPTRANSIT_50',
       'SHAPE'],
      dtype='object')

In [10]:
#if results geodatabase doesn't exist, create it
print("Checking if AccessToOpportunities exists...")
if not arcpy.Exists(os.path.join(dirResults, "AccessToOpportunities.gdb")):
    print("Creating AccessToOpportunities...")
    arcpy.management.CreateFileGDB(dirResults, "AccessToOpportunities.gdb")
else:
    print("AccessToOpportunities exists...")
#print(AnalysisAreasInput)  

sdfTAZATOforWFRCDataPortal.spatial.to_featureclass('results/AccessToOpportunities.gdb/ATO',sanitize_columns=False)

Checking if AccessToOpportunities exists...
AccessToOpportunities exists...


'e:\\GitHub\\ATO-Web-App\\_dataprep\\results\\AccessToOpportunities.gdb\\ATO'